In [3]:
%%capture
import math
import numpy as np
import pandas as pd
import nltk 
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="https://raw.githubusercontent.com/liraop/recinfo_lab2/master/data/results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)
documents = data.text.count()
N = documents 

def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    year_pattern = rpt(r'\d{4}')
    
    patterns = [word_pattern, year_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)
    return words


def get_idf(index):
    for ngram in index:
        k = len(index[ngram])
        m = documents + 1
        index[ngram]['idf'] = math.log(m/k)
         

def build_index(dataset):
    document_index = 0
    index = {}
    
    for entry in dataset.text:
        document_index = document_index + 1
            
        for ngram in parse(entry):
                if ngram in index: #is ngram already on index?
                    if document_index in index[ngram]: # is it in the same document?
                        index[ngram][document_index] = index[ngram][document_index] + 1                        
                    else: # nope
                        index[ngram][document_index] = 1 
                else: # no, sir
                    index[ngram] = {document_index: 1}
    get_idf(index)           
    
    return index
  
def get_top10rank(doc_score):
   
    df_tmp = pd.DataFrame(doc_score.items(), columns=["document", "score"])
    df_tmp['r']= df_tmp.score.rank(ascending=False, method="first")
    df_tmp.sort_values("r", inplace = True)
    df_tmp = df_tmp[:10]
        
    return df_tmp
    
index = build_index(data)

def bin_query_vector(index, query):
    query_vector = []
    
    for word in index:
        if word in query.split():
            query_vector.append(1)
        else:
            query_vector.append(0)
            
    return query_vector

def bin_document_vector(index):
    document_vector = []
    
    for doc_id in range(1,documents+1):
        doc_vec = []
        for ngram in index:
            if doc_id in index[ngram].keys():
                doc_vec.append(1)
            else:
                doc_vec.append(0)
                
        document_vector.append(doc_vec)
    
    return document_vector
                

def f_bin(query_vector, doc_vector):
    rec = {}

    for doc_id in range(len(doc_vector)):
        sim = 0
        vector = doc_vector[doc_id]
        for i in range(len(vector)):
            sim += (query_vector[i] * vector[i])
        rec[doc_id+1] = sim
    
    return rec

def binary_vsm(index, query):
    query_vector = bin_query_vector(index, query)   
    doc_vector = bin_document_vector(index)
    
    return f_bin(query_vector, doc_vector)

def tf_document_vector(index):
    document_vector = []
    
    for doc_id in range(1,documents+1):
        doc_vec = []
        for ngram in index:
            if doc_id in index[ngram].keys():
                y = index[ngram][doc_id]
                doc_vec.append(y)
            else:
                doc_vec.append(0)
                
        document_vector.append(doc_vec)
        
    return document_vector

def tf_query_vector(index, query):
    query_vector = []
    
    for ngram in index:
        w = 0
        for term in query.split():
            if ngram == term:
                w += 1
        query_vector.append(w)
        
    return query_vector

def f_tf(query_vector, doc_vector):
    rec = {}
    for doc_id in range(len(doc_vector)):
        sim = 0
        vector = doc_vector[doc_id]
        for i in range(len(vector)):
            sim += (query_vector[i] * vector[i])
        rec[doc_id+1] = sim
    return rec

def tf_vsm(index, query):
    query_vector = tf_query_vector(index, query)   
    doc_vector = tf_document_vector(index)
    
    return f_tf(query_vector, doc_vector)

def tfidf_document_vector(index):
    document_vector = []
    
    for doc_id in range(1,documents+1):
        doc_vec = []
        for ngram in index:
            if doc_id in index[ngram].keys():
                y = index[ngram][doc_id] * index[ngram]['idf']
                doc_vec.append(y)
            else:
                doc_vec.append(0)
                
        document_vector.append(doc_vec)
        
    return document_vector


def tfidf_vsm(index, query):
    query_vector = tf_query_vector(index, query)   
    doc_vector = tfidf_document_vector(index)
    
    return f_tf(query_vector, doc_vector)


def f_bm25(query_vector, doc_vector, k):
    rec = {}
    for doc_id in range(len(doc_vector)):
        sim = 0
        vector = doc_vector[doc_id]
        for i in range(len(vector)):
            if vector[i] != 0:
                y = (k+1) * query_vector[i]
                dom = (query_vector[i] * y)/(query_vector[i]+k)
                sim += (dom * math.log10((documents + 1)/vector[i]))
        rec[doc_id+1] = sim
    return rec
    

def bm25_vsm(index, query, k):
    query_vector = tf_query_vector(index, query)   
    doc_vector = tf_document_vector(index)
    
    return f_bm25(query_vector, doc_vector, k)


### Q1: Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento. Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank ( 10 pts)

In [5]:
chosen_one = 13
doc_url = "https://brasil.elpais.com/brasil/2019/03/15/cultura/1552681746_926411.html"
query = "Gabo colombiano solidão"

def reciprocal_rank(rank, selected_doc):
    position = 0
    for index, row in rank.iterrows():
        position += 1
        if row["document"] == chosen_one:
            return 1.0/position
%%capture        
rank_binario = get_top10rank(binary_vsm(index, query))
print(rank_binario)
print("Reciprocal rank:", reciprocal_rank(rank_binario, chosen_one))

     document  score     r
12         13      2   1.0
3           4      1   2.0
62         63      1   3.0
64         65      1   4.0
156       157      1   5.0
181       182      1   6.0
183       184      1   7.0
187       188      1   8.0
0           1      0   9.0
1           2      0  10.0
Reciprocal rank: 1.0


#### Vemos acima que o reciprocal rank retorna o documento com maior score retornado pelo vsm. Está correto pois o documento, além de ser o escolhido, tem o maior numero de termos da query nele.

In [6]:
rank_tf = get_top10rank(tf_vsm(index, query))
print(rank_tf)
print("Reciprocal rank:", reciprocal_rank(rank_tf, chosen_one))

     document  score     r
12         13      4   1.0
62         63      2   2.0
3           4      1   3.0
64         65      1   4.0
156       157      1   5.0
181       182      1   6.0
183       184      1   7.0
187       188      1   8.0
0           1      0   9.0
1           2      0  10.0
Reciprocal rank: 1.0


In [7]:
rank_tfidf = get_top10rank(tfidf_vsm(index, query))
print(rank_tfidf)
print("Reciprocal rank:", reciprocal_rank(rank_tfidf, chosen_one))

     document      score     r
12         13  18.620108   1.0
62         63   8.270333   2.0
3           4   4.828314   3.0
156       157   4.422849   4.0
181       182   4.422849   5.0
183       184   4.422849   6.0
64         65   4.135167   7.0
187       188   4.135167   8.0
0           1   0.000000   9.0
1           2   0.000000  10.0
Reciprocal rank: 1.0


In [9]:
rank_bm25 = get_top10rank(bm25_vsm(index, query, 10))
print(rank_bm25)
print("Reciprocal rank:", reciprocal_rank(rank_bm25, chosen_one))

     document     score     r
12         13  4.318759   1.0
3           4  2.397940   2.0
64         65  2.397940   3.0
156       157  2.397940   4.0
181       182  2.397940   5.0
183       184  2.397940   6.0
187       188  2.397940   7.0
62         63  2.096910   8.0
0           1  0.000000   9.0
1           2  0.000000  10.0
Reciprocal rank: 1.0


#### O mesmo ocorre com os outros modelos. 

### Q2: A partir do gabarito fornecido em OBS1, calcule o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. Para os cálculos do MAP, considere que um documento é relevante para uma dada consulta se este documento estiver entre os documentos do gabarito para essa consulta, senão ele deve ser considerado irrelevante. (10 pts)

### Q3: Repita Q2 usando a avaliação multi-nível DCG. Utilize o campo "level" do gabarito para o cálculo do DCG e do idealDCG. (10 pts)